<a href="https://colab.research.google.com/github/ASDF44/ASDF44.github.io/blob/master/image2RGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd 
from google.colab import drive
drive.mount('/content/drive/')
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
data = pd.read_csv('/content/drive/My Drive/image2rgb/handwritten-data.csv')

print(data.shape)

data.rename(columns={'0':'label'}, inplace=True)
print(data.tail())

(372450, 785)
        label  0.1  0.2  0.3  0.4  ...  0.644  0.645  0.646  0.647  0.648
372445     25    0    0    0    0  ...      0      0      0      0      0
372446     25    0    0    0    0  ...      0      0      0      0      0
372447     25    0    0    0    0  ...      0      0      0      0      0
372448     25    0    0    0    0  ...      0      0      0      0      0
372449     25    0    0    0    0  ...      0      0      0      0      0

[5 rows x 785 columns]


In [0]:
idx = [i for i in range(26)]
rgb = [[255,0,0], [0,0,255], [255,255,0], [0,0,255], [0,255,0], [0,255,0], 
 [0,255,0], [255,128,0], [255,255,255], [255,128,0], [255,128,0],
 [255,255,0], [255,0,0], [255,128,0],[255,255,255], [204,0,204],
 [204,0,204], [255,0,0], [255,255,0], [0,0,255], [255,128,0], [204,0,204],
 [0,0,255],[0,0,0], [255,255,0], [0,0,0]]
for i in range(len(rgb)):
    for j in range(len(rgb[i])):
        if rgb[i][j] == 0:
            rgb[i][j] = 0
        elif rgb[i][j] == 255:
            rgb[i][j] = 1
        else:
            rgb[i][j] = 0.5
rgbmappings = dict(zip(idx, rgb))
rgbmappings

{0: [1, 0, 0],
 1: [0, 0, 1],
 2: [1, 1, 0],
 3: [0, 0, 1],
 4: [0, 1, 0],
 5: [0, 1, 0],
 6: [0, 1, 0],
 7: [1, 0.5, 0],
 8: [1, 1, 1],
 9: [1, 0.5, 0],
 10: [1, 0.5, 0],
 11: [1, 1, 0],
 12: [1, 0, 0],
 13: [1, 0.5, 0],
 14: [1, 1, 1],
 15: [0.5, 0, 0.5],
 16: [0.5, 0, 0.5],
 17: [1, 0, 0],
 18: [1, 1, 0],
 19: [0, 0, 1],
 20: [1, 0.5, 0],
 21: [0.5, 0, 0.5],
 22: [0, 0, 1],
 23: [0, 0, 0],
 24: [1, 1, 0],
 25: [0, 0, 0]}

In [0]:
data['red'] = [rgbmappings[i][0] for i in data['label']]
data['green'] = [rgbmappings[i][1] for i in data['label']]
data['blue'] = [rgbmappings[i][2] for i in data['label']]
data.head()

,label,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,...,0.612,0.613,0.614,0.615,0.616,0.617,0.618,0.619,0.620,0.621,0.622,0.623,0.624,0.625,0.626,0.627,0.628,0.629,0.630,0.631,0.632,0.633,0.634,0.635,0.636,0.637,0.638,0.639,0.640,0.641,0.642,0.643,0.644,0.645,0.646,0.647,0.648,red,green,blue
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0


In [0]:
X = data.drop(['label' ,'red', 'green', 'blue'],axis = 1)
X.head()

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.40,...,0.609,0.610,0.611,0.612,0.613,0.614,0.615,0.616,0.617,0.618,0.619,0.620,0.621,0.622,0.623,0.624,0.625,0.626,0.627,0.628,0.629,0.630,0.631,0.632,0.633,0.634,0.635,0.636,0.637,0.638,0.639,0.640,0.641,0.642,0.643,0.644,0.645,0.646,0.647,0.648
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
!pip install keras_tqdm
from keras_tqdm import TQDMNotebookCallback
y = data['red']
y.head()

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: red, dtype: float64

In [0]:
(X_train, X_test, Y_train, Y_test) = train_test_split(X, y)
standard_scaler = MinMaxScaler()
standard_scaler.fit(X_train)

X_train = standard_scaler.transform(X_train)
X_test = standard_scaler.transform(X_test)

X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32')
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32')

print(X_train.shape)

Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

print(Y_test.shape)

num_classes = Y_test.shape[1]

model = Sequential()

model.add(Conv2D(64, (5, 5), input_shape=(28, 28, 1), activation='relu', data_format="channels_last", padding="same"))
model.add(Conv2D(64, (5, 5), input_shape=(28, 28, 1), activation='relu', data_format="channels_last", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', data_format="channels_last", padding="same"))
model.add(Conv2D(128, (3, 3), activation='relu', data_format="channels_last", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=20, batch_size=256, verbose=2, callbacks=[TQDMNotebookCallback()])
# Final evaluation of the model
scores = model.evaluate(X_test,Y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))
model.save('/content/drive/My Drive/image2rgb/models/red/weightsR.model')
model.save_weights("/content/drive/My Drive/image2rgb/models/red/modelR.h5")
model_json = model.to_json()
import warnings
warnings.filterwarnings("ignore")
with open("/content/drive/My Drive/image2rgb/models/red/modelR.json", "w") as json_file:
    json_file.write(model_json)

(279337, 28, 28, 1)
(93113, 2)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 279337 samples, validate on 93113 samples


Epoch 1/20







 - 5434s - loss: 0.1007 - acc: 0.9606 - val_loss: 0.0331 - val_acc: 0.9890
Epoch 2/20


KeyboardInterrupt: ignored

In [0]:
y = data['green']
y.head()

In [0]:
import warnings
warnings.filterwarnings("ignore")
(X_train, X_test, Y_train, Y_test) = train_test_split(X, y)
standard_scaler = MinMaxScaler()
standard_scaler.fit(X_train)

X_train = standard_scaler.transform(X_train)
X_test = standard_scaler.transform(X_test)

X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32')
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32')

print(X_train.shape)

Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

print(Y_test.shape)
num_classes = Y_test.shape[1]

model = Sequential()

model.add(Conv2D(64, (5, 5), input_shape=(28, 28, 1), activation='relu', data_format="channels_last", padding="same"))
model.add(Conv2D(64, (5, 5), input_shape=(28, 28, 1), activation='relu', data_format="channels_last", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', data_format="channels_last", padding="same"))
model.add(Conv2D(128, (3, 3), activation='relu', data_format="channels_last", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=20, batch_size=256, verbose=2, callbacks=[TQDMNotebookCallback()])

# Final evaluation of the model
scores = model.evaluate(X_test,Y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))
model.save('/content/drive/My Drive/image2rgb/weightsG.model')
model.save_weights("/content/drive/My Drive/image2rgb/modelG.h5")
model_json = model.to_json()
with open("/content/drive/My Drive/image2rgb/modelG.json", "w") as json_file:
    json_file.write(model_json)

In [0]:
!pip install keras_tqdm
from keras_tqdm import TQDMNotebookCallback
y = data['blue']
y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: blue, dtype: float64

In [0]:
import warnings
warnings.filterwarnings("ignore")
(X_train, X_test, Y_train, Y_test) = train_test_split(X, y)
standard_scaler = MinMaxScaler()
standard_scaler.fit(X_train)

X_train = standard_scaler.transform(X_train)
X_test = standard_scaler.transform(X_test)

X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32')
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32')

print(X_train.shape)

Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

print(Y_test.shape)

num_classes = Y_test.shape[1]

model = Sequential()

model.add(Conv2D(64, (5, 5), input_shape=(28, 28, 1), activation='relu', data_format="channels_last", padding="same"))
model.add(Conv2D(64, (5, 5), input_shape=(28, 28, 1), activation='relu', data_format="channels_last", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', data_format="channels_last", padding="same"))
model.add(Conv2D(128, (3, 3), activation='relu', data_format="channels_last", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=20, batch_size=256, verbose=2, callbacks=[TQDMNotebookCallback()])


(279337, 28, 28, 1)
(93113, 2)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 279337 samples, validate on 93113 samples


Epoch 1/20







 - 506s - loss: 0.0586 - acc: 0.9791 - val_loss: 0.0184 - val_acc: 0.9944
Epoch 2/20


 - 505s - loss: 0.0174 - acc: 0.9949 - val_loss: 0.0174 - val_acc: 0.9947
Epoch 3/20


 - 507s - loss: 0.0131 - acc: 0.9962 - val_loss: 0.0123 - val_acc: 0.9964
Epoch 4/20


...

 - 509s - loss: 0.0024 - acc: 0.9993 - val_loss: 0.0052 - val_acc: 0.9988
Epoch 18/20


 - 509s - loss: 0.0019 - acc: 0.9995 - val_loss: 0.0085 - val_acc: 0.9985
Epoch 19/20


 - 509s - loss: 0.0022 - acc: 0.9994 - val_loss: 0.0075 - val_acc: 0.9986
Epoch 20/20


 - 510s - loss: 0.0023 - acc: 0.9993 - val_loss: 0.0058 - val_acc: 0.9987

CNN Error: 0.13%


OSError: ignored

In [0]:
# Final evaluation of the model
scores = model.evaluate(X_test,Y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))
model.save('/content/drive/My Drive/image2rgb/weightsB.model')
model.save_weights("/content/drive/My Drive/image2rgb/modelB.h5")
model_json = model.to_json()
with open("/content/drive/My Drive/image2rgb/modelB.json", "w") as json_file:
    json_file.write(model_json)


CNN Error: 0.13%


In [0]:
print("CNN Error: %.2f%%" % (100-colorscores[1]*100))

CNN Error: 0.79%


In [0]:
print("CNN Error: %.2f%%" % (100-labelscores[1]*100))

CNN Error: 0.13%
